# Sudoku Solver

In [1]:
#Running this cell displays a button to toggle hidden code
#From: http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/

from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

This program takes text files of 9x9 grids of numbers as input, where a 0 represents a blank square. See https://projecteuler.net/problem=96 for sample puzzles.

**Note: To use this notebook, please run the cells in order. Using "Run All" will not work.**

Currently, this solver consists of four functions:

**OnePossible():**

This function checks each section (rows, columns and boxes) to see if there are any squares that only have one possible number they could be. For example, if the row containing the square had the numbers 1 through 5, and the column containing the square had the numbers 6 through 8, this function would find that the only possible choice is 9. It would then make that move and update the puzzle as well as the options for each empty square.

**OnlyPossible():**

This function checks if a number can only be in one square within a section (row, column or box). For example, say there are 3 spaces left within a box. The possibilities for the first are 1, 2 and 3, and the possibilities for
both the second and third squares are 1 and 2. This algorithm would find that 3 is only possible in the first square, and perform that move.

**NakedGroups():**

This function checks if there are any groups of _n_ squares in a section (row, column, or box) that have the same _n_ options available. This is called a naked group. Then it removes these _n_ options from every other square in the section. For example, if the only options of two squares in a row are 1 and 2, we know that 1 and 2 must go in these squares and not any of the other squares in the row.

**Bifurcate():**

This function finds a square with the minimum number of options and tries each possible option in a copy version of the puzzle. Copy puzzles that result in conflicts are disregarded, and ones that do not are kept. If one bifurcation does not solve the puzzle, the resulting copied puzzles will be bifurcated again, recursively.

### The following two cells contain the functions necessary for the notebook to run, please run them first.

In [2]:
#Widget for selecting a file
#https://codereview.stackexchange.com/questions/162920/file-selection-button-for-jupyter-notebook

import traitlets
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog


class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True)

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [3]:
#SUDOKU SOLVER FUNCTIONS


def GenerateOptions(r, c, puzzle):
    
    """Input: (row number, column number, puzzle board)
    Output: One-dimensional list of integer options
    
    Returns a list of possible numbers for an empty square, given a pair of coordinates.
    If a number is already printed, returns an empty list."""
    
    #Generate full list of options 1-9:
    o_list = [x for x in range(1,10)]
    
    #If square is not empty, remove all options:
    if puzzle[r][c] != 0:
        o_list = []
    
    #Continue to refine options if the square is empty:
    else:
        
        #Check row, remove all numbers from the options that exist in that row:
        for y in range(0,9):
            if puzzle[r][y] in o_list:
                o_list.remove(puzzle[r][y])


        #Check col, remove all numbers from the options that exist in that col:
        for x in range(0,9):
            if puzzle[x][c] in o_list:
                o_list.remove(puzzle[x][c])

        #Check box, remove all numbers from the options that exist in that box:
        #Boxes numbered 1-9 left to right, top to bottom

        #Box 1
        if r < 3 and c < 3:
            for x in range(0,3):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 2
        elif r < 3 and 2 < c < 6:
            for x in range(0,3):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 3
        elif r < 3 and c > 5:
            for x in range(0,3):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 4
        elif 2 < r < 6 and c < 3:
            for x in range(3,6):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 5
        elif 2 < r < 6 and 2 < c < 6:
            for x in range(3,6):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 6
        elif 2 < r < 6 and c > 5:
            for x in range(3,6):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 7
        elif r > 5 and c < 3:
            for x in range(6,9):
                for y in range(0,3):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 8
        elif r > 5 and 2 < c < 6:
            for x in range(6,9):
                for y in range(3,6):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        #Box 9
        elif r > 5 and c > 5:
            for x in range(6,9):
                for y in range(6,9):
                    if puzzle[x][y] in o_list:
                        o_list.remove(puzzle[x][y])

        else:
            print("ERROR")
        
                              
    return o_list
    
    

def MakeMove(r, c, v, puzzle, options):
    
    """Input: (row number, column number, value, puzzle board, options list)
    Output: [puzzle board, options list]
    
    Updates the board at row r and col c with value v, returns an updated board and options. Updates all affected options lists."""
    
    #Creating an empty 9x9 list and copying options to it (Note to self: Need to be WAY more careful about copying lists)
    options_copy = [[] for x in range(0,9)]
    for y in range(0,9):
        options_copy[y] = [[] for x in range(0,9)]
    
    for x in range(0, len(options)):
        for y in range(0, len(options[x])):
            for z in range(0, len(options[x][y])):   
                options_copy[x][y].append(options[x][y][z])
    
    #Making the move:
    puzzle[r][c] = v
                              
    #Updating the affected options:
    
    #Affected square:
    options_copy[r][c] = []
    
    #Affected row:
    for y in range(0,9):
        if v in options_copy[r][y]:
            options_copy[r][y].remove(v)
        
    #Affected col:
    for x in range(0,9):
        if v in options_copy[x][c]:
            options_copy[x][c].remove(v)
        
    #Affected box:
    #Boxes numbered 1-9 left to right, top to bottom
    
    #Box 1
    if r < 3 and c < 3:
        for x in range(0,3):
            for y in range(0,3):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
        
    #Box 2
    elif r < 3 and 2 < c < 6:
        for x in range(0,3):
            for y in range(3,6):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 3
    elif r < 3 and c > 5:
        for x in range(0,3):
            for y in range(6,9):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
        
    #Box 4
    elif 2 < r < 6 and c < 3:
        for x in range(3,6):
            for y in range(0,3):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 5
    elif 2 < r < 6 and 2 < c < 6:
        for x in range(3,6):
            for y in range(3,6):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 6
    elif 2 < r < 6 and c > 5:
        for x in range(3,6):
            for y in range(6,9):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 7
    elif r > 5 and c < 3:
        for x in range(6,9):
            for y in range(0,3):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 8
    elif r > 5 and 2 < c < 6:
        for x in range(6,9):
            for y in range(3,6):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    #Box 9
    elif r > 5 and c > 5:
        for x in range(6,9):
            for y in range(6,9):
                if v in options_copy[x][y]:
                    options_copy[x][y].remove(v)
    
    else:
        print("ERROR")
        
    result = [puzzle, options_copy]
    
    return result
  
    

def OnePossible(puzzle, options):
    
    """Input: (puzzle board, options list)
    Output: [puzzle board, options list, move count]
    
    Checks all squares to see if they only have one option, then makes that move. Returns new board, new options, and the count of moves made during the function call."""
    
    move_count = 0
    
    #For each square:
    for x in range(0,9):
        for y in range(0,9):
            
            #If there is only one option, make that move and add 1 to the move count:
            if len(options[x][y]) == 1:
                
                new_state = MakeMove(x, y, options[x][y][0], puzzle, options)
                puzzle = new_state[0]
                options = new_state[1]
                move_count += 1
    
    result = [puzzle, options, move_count]
    
    return result



def OnlyPossible(puzzle, options):

    """Input: (puzzle board, options list)
    Output: [puzzle board, options list, move count]
    
    Checks all squares to see if a number can only be in one square within a section (row, column, or box), then makes that move.
    Returns new board, new options, and the count of moves made during the function call."""
    
    move_count = 0
    
    #For every square:
    for x in range(0,9):
        for y in range(0,9):
            
            #If this square is empty:
            if puzzle[x][y] == 0:     
            
                #For every option for that square:
                for v in options[x][y]:

                    make_move = True

                    #Check if this option is available to any other square in the row, (except the one in question):
                    for col in range(0,9):
                        if v in options[x][col] and col != y:
                            make_move = False
                            
                    #Check if this option is available to any other square in the col:
                    for row in range(0,9):
                        if v in options[row][y] and row != y:
                            make_move = False
                            
                    #Check if this option is available to any other square in the box:
                    #Box 1
                    if x < 3 and y < 3:
                        for row in range(0,3):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 2
                    elif x < 3 and 2 < y < 6:
                        for row in range(0,3):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 3
                    elif x < 3 and y > 5:
                        for row in range(0,3):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 4
                    elif 2 < x < 6 and y < 3:
                        for row in range(3,6):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 5
                    elif 2 < x < 6 and 2 < y < 6:
                        for row in range(3,6):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 6
                    elif 2 < x < 6 and y > 5:
                        for row in range(3,6):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 7
                    elif x > 5 and y < 3:
                        for row in range(6,9):
                            for col in range(0,3):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 8
                    elif x > 5 and 2 < y < 6:
                        for row in range(6,9):
                            for col in range(3,6):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    #Box 9
                    elif x > 5 and y > 5:
                        for row in range(6,9):
                            for col in range(6,9):
                                if v in options[row][col]:
                                    if row != x and col !=y:
                                        make_move = False

                    else:
                        print("ERROR")
                    
                    
                    #If all checks have been passed, make move and add to move count:
                    if make_move == True:
                        new_state = MakeMove(x, y, v, puzzle, options)
                        puzzle = new_state[0]
                        options = new_state[1]
                        move_count += 1

    result = [puzzle, options, move_count]
                        
    return result


#FUNCTION: NakedGroups(puzzle, options)
#Checks for pairs in a section (row, column or box) that contain the same two numbers as options, ex. 1,2 and 1,2.
#1 and 2 can then be removed from all other options lists in that section, since 1 and 2 must go in those squares.
#Returns updated options
def NakedGroups(puzzle, options):
    
    """Input: (puzzle board, options list)
    Output: [puzzle board, options list, move count]
    
    Checks for pairs in a section (row, column or box) that contain the same two numbers as options, ex. 1,2 and 1,2.
    1 and 2 can then be removed from all other options lists in that section, since 1 and 2 must go in those squares.
    Returns the updated options list."""
    
    #For each row:
    for r in range(0,9):
        
        for c in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Check if square's options are equal to any other square in that row (if it appears after c):
            for c2 in range(c+1,9):
                if c2 != c and options[r][c] == options[r][c2]:
                    
                    #Add [r,c] to the list if it is the first time an equal has been found:
                    if equal_count == 1:
                        equal_list.append([r,c])
                        
                    #Add [r,c2] if equal and add to the count:
                    equal_list.append([r,c2])
                    equal_count += 1
                    
            #Equal_list is complete for square [r,c] for row r
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:
                
                #A naked group has been found, remove every option in options[r][c] from every square in the row:
                for option in options[r][c]:
                    for c3 in range(0,9):
                        
                        #Checks if the square checked is not in the naked group:
                        in_group = False
                        for ng_square in equal_list:
                            if c3 == ng_square[1]:
                                in_group = True
                                
                        #If the square is not in the group, remove the options:
                        if in_group == False:  
                            if option in options[r][c3]:
                                options[r][c3].remove(option)
            
                    
    
    #For each col:
    for c in range(0,9):
        
        for r in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Check if square's options are equal to any other square in that col (if it appears after r):
            for r2 in range(r+1,9):
                if r2 != r and options[r][c] == options[r2][c]:
                    
                    #Add [r,c] to the list if it is the first time an equal has been found:
                    if equal_count == 1:
                        equal_list.append([r,c])
                        
                    #Add [r2,c] if equal and add to the count:
                    equal_list.append([r2,c])
                    equal_count += 1
                    
            #Equal_list is complete for square [r,c] for col c
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:
                
                #A naked group has been found, remove every option in options[r][c] from every square in the col:
                for option in options[r][c]:
                    for r3 in range(0,9):
                        
                        #Checks if the square checked is not in the naked group:
                        in_group = False
                        for ng_square in equal_list:
                            if r3 == ng_square[0]:
                                in_group = True
                                
                        #If the square is not in the group, remove the options:
                        if in_group == False:  
                            if option in options[r3][c]:
                                options[r3][c].remove(option)
    
    #For each box:
    for r in range(0,9):
        for c in range(0,9):
            
            #List of squares that have equal options, and a count of how many squares have equal options:
            #(Resets variables for the next loop)
            equal_list = []
            equal_count = 1
            
            #Box 1
            if r < 3 and c < 3:                
                startx = 0
                endx = 3
                starty = 0
                endy = 3                            

            #Box 2
            elif r < 3 and 2 < c < 6:                
                startx = 0
                endx = 3
                starty = 3
                endy = 6

            #Box 3
            elif r < 3 and c > 5:                
                startx = 0
                endx = 3
                starty = 6
                endy = 9

            #Box 4
            elif 2 < r < 6 and c < 3:
                startx = 3
                endx = 6
                starty = 0
                endy = 3

            #Box 5
            elif 2 < r < 6 and 2 < c < 6:
                startx = 3
                endx = 6
                starty = 3
                endy = 6

            #Box 6
            elif 2 < r < 6 and c > 5:
                startx = 3
                endx = 6
                starty = 6
                endy = 9

            #Box 7
            elif r > 5 and c < 3:
                startx = 6
                endx = 9
                starty = 0
                endy = 3

            #Box 8
            elif r > 5 and 2 < c < 6:
                startx = 6
                endx = 9
                starty = 3
                endy = 6

            #Box 9
            elif r > 5 and c > 5:
                startx = 6
                endx = 9
                starty = 6
                endy = 9
                
            else:
                print("Error")
                    
            #Check if square's options are equal to any other square in that box:
            for x in range(startx,endx):
                for y in range(starty,endy):
                    if not(x == r and y == c) and options[r][c] == options[x][y]:

                        #Add [r,c] to the list if it is the first time an equal has been found:
                        if equal_count == 1:
                            equal_list.append([r,c])

                        #Add [x,y] if equal and add to the count:
                        equal_list.append([x,y])
                        equal_count += 1

            #Equal_list is complete for square [r,c]
            #If an equal was found, check that the number of equals found is equal to the number of options for [r,c]
            if equal_count == len(options[r][c]) and equal_count > 1:

                #A naked group has been found, remove every option in options[r][c] from every square in the box:
                for option in options[r][c]:
                    for x2 in range(startx,endx):
                        for y2 in range(starty,endy):

                            #Checks if the square checked is not in the naked group:
                            in_group = False
                            for ng_square in equal_list:
                                if x2 == ng_square[0] and y2 == ng_square[1]:
                                    in_group = True

                            #If the square is not in the group, remove the options:
                            if in_group == False:  
                                if option in options[x2][y2]:
                                    options[x2][y2].remove(option)
                                    
    return options
                                    
                                    
                                    
def ErrorCheck(board):
    
    """Input: (puzzle board, options list)
    Output: True or False
    
    Takes a puzzle board (the current board or a copy through the bifurcate function) and checks for any conflicts.
    Returns True if a board has no conflicts, returns False if it does."""
    
    no_errors = True
    errorcount = 0
    
    #Check rows:
    for r in range(0,9):
        #For each number 1-9, go through each square in the row and counts how many of that number are in that section
        for num in range(1,10):
            
            numcount = 0
            
            for c in range(0,9):
                
                if board[r][c] == num:
                    
                    numcount += 1
                    
            #If a duplicate was found in the row, add the number of duplicates to the errorcount and updates no_errors: 
            if numcount > 1:
                
                errorcount = errorcount + (numcount - 1)
                no_errors = False
                
    #Check cols:
    for c in range(0,9):
        #For each number 1-9, go through each square in the col and counts how many of that number are in that section
        for num in range(1,10):
            
            numcount = 0
            
            for r in range(0,9):
                
                if board[r][c] == num:
                    
                    numcount += 1
                    
            #If a duplicate was found in the col, add the number of duplicates to the errorcount and updates no_errors: 
            if numcount > 1:
                
                errorcount = errorcount + (numcount - 1)
                no_errors = False
                
    #Check boxes:
    for box in range(1,10):
        
        #Sets the appropriate range of squares to check for each box:
        #Box 1
        if box == 1:                
            startx = 0
            endx = 3
            starty = 0
            endy = 3                            

        #Box 2
        elif box == 2:                
            startx = 0
            endx = 3
            starty = 3
            endy = 6

        #Box 3
        elif box == 3:                
            startx = 0
            endx = 3
            starty = 6
            endy = 9

        #Box 4
        elif box == 4:
            startx = 3
            endx = 6
            starty = 0
            endy = 3

        #Box 5
        elif box == 5:
            startx = 3
            endx = 6
            starty = 3
            endy = 6

        #Box 6
        elif box == 6:
            startx = 3
            endx = 6
            starty = 6
            endy = 9

        #Box 7
        elif box == 7:
            startx = 6
            endx = 9
            starty = 0
            endy = 3

        #Box 8
        elif box == 8:
            startx = 6
            endx = 9
            starty = 3
            endy = 6

        #Box 9
        elif box == 9:
            startx = 6
            endx = 9
            starty = 6
            endy = 9

        else:
            print("Error")
            
        #Run through the squares specified to check for duplicates.
        #For each number:
        for num in range(1,10):
            
            numcount = 0
            
            #For each row index in the box:
            for r in range(startx, endx):

                #For each col index in the box
                for c in range(starty,endy):

                    if board[r][c] == num:

                        numcount += 1

                #If a duplicate was found in the col, adds the number of duplicates to the errorcount and updates no_errors: 
                if numcount > 1:

                    errorcount = errorcount + (numcount - 1)
                    no_errors = False
    
    
    return no_errors


#FUNCTION: CountUnsolved(puzzle)
#Takes a puzzle and returns the number of unsolved squares
def CountUnsolved(board):
    
    unsolved_count = 0
    
    for r in range(0,9):
        for c in range(0,9):
            if board[r][c] == 0:
                
                unsolved_count += 1
                
    return unsolved_count
                
                                    

def FindBifurcation(board_fb, options_fb, userinput_fb):
    
    """Input: (puzzle board, options list, True/False)
    Output: [row number, column number]
    
    Finds the best squares to bifurcate, among the squares with the minimum amount of options.
    If userinput is True, will print the current board, the possible squares to bifurcate, and allow the user to choose a square."""
    
    possiblesquares = []
    options_number = 0
    chosen_sq = []
        
    #Looks for any squares with 2 options, then 3 options ... until 9
    for num in range(2,10):
        
        #Only continues to look if no possible squares have been found
        if options_number == 0:
            
            for r in range(0,9):
                for c in range(0,9):
                    
                    #If there are (num) options for the current square checked:
                    if len(options_fb[r][c]) == num:
                        
                        #Record the minimun number of options found, and add [r,c] to the list of possible squares to bifurcate:
                        options_number = num
                        possiblesquares.append([r,c])
     
    if len(possiblesquares) != 0:
    
        #Chooses the first square in the list to bifurcate:
        chosen_sq = possiblesquares[0]

        #List of possiblesquares is complete
        #If the user wants to choose a square to bifurcate, the following runs:
        if userinput_fb == True:

            print(f"These squares have {options_number} options:")
            print()
            print(possiblesquares)
            print()
            print("The first and second coordinates are the row and column numbers counted from 0 to 8, respectively.")
            print()
            print("Please enter the number of the square you wish to bifurcate from.")
            print("For example, enter 2 if you want to bifurcate the second square in the list.")
            sq_index = int(input("Which square do you want to bifurcate? ")) - 1
            print()

            chosen_sq = possiblesquares[sq_index]

            print(f"You have chosen to bifurcate square {chosen_sq}.")
        
    return chosen_sq            
        

#FUNCTION: Bifurcate()
#Takes the coordinates row and col for a square with n options
#Then makes n copies of the board and solves them using the other functions
#Checks for inconsistencies and returns the correct board
#Works, recursively: returns solved boards in format: solved = [(board1, options1), (board2, options2) ...]
def Bifurcate(row, col, puzzle, options):
    
    #Number of copy boards to make is the same as the number of options for the square
    copy_num = len(options[row][col])
    
    copy_boards = []
    copy_options = []
    
    #Copies the puzzle and options as many times as we will bifurcate:
    #copy_boards and copy_options are in format [copynumber][row][col]
    for n in range(0, copy_num):
        
        #Creating an empty 9x9 list and copying puzzle to it
        puzzle_copy = [[] for x in range(0,9)]
        for y in range(0,9):
            puzzle_copy[y] = [None for x in range(0,9)]

        for x in range(0, len(puzzle)):
            for y in range(0, len(puzzle[x])):   
                    puzzle_copy[x][y] = puzzle[x][y]
        
        #Creating an empty 9x9 list and copying options to it
        options_copy = [[] for x in range(0,9)]
        for y in range(0,9):
            options_copy[y] = [[] for x in range(0,9)]

        for x in range(0, len(options)):
            for y in range(0, len(options[x])):
                for z in range(0, len(options[x][y])):   
                    options_copy[x][y].append(options[x][y][z])
        
        copy_boards.append(puzzle_copy)
        copy_options.append(options_copy)
    
    #Makes a different move in the target square on each copied puzzle:
    for n in range(0, len(copy_boards)):
        
        mresult = MakeMove(row, col, options[row][col][n], copy_boards[n], copy_options[n])
        copy_boards[n] = mresult[0]
        copy_options[n] = mresult[1]
    
    #Runs all the other solving algorithms on all of the bifurcated puzzles:
    for n in range(0, len(copy_boards)):
        
        sresult = SimpleSolve(copy_boards[n], copy_options[n])
        copy_boards[n] = sresult[0]
        copy_options[n] = sresult[1]
        
    #Checks for inconsistencies in the boards and throws out ones that have any:
    for n in range(0, len(copy_boards)): 
        
        no_errors = ErrorCheck(copy_boards[n])
        
        if no_errors == False:
            del copy_boards[n]
            del copy_options[n]
    
    result_boards = []
    result_options = []
    
    #Adds all solved boards to the results list, and removes them from the copy lists:
    ind = 0
    while ind < len(copy_boards):
        
        solve_check = CountUnsolved(copy_boards[ind])
        
        #If board is solved, add to results and remove:
        if solve_check == 0:
            
            result_boards.append(copy_boards.pop(ind))
            result_options.append(copy_options.pop(ind))
            ind -= 1
            
        ind += 1
            
            
    #Recursively calls Bifurcate again for any boards left unsolved without inconsistencies:
    for n in range(len(copy_boards)):
            
        b_square = FindBifurcation(copy_boards[n], copy_options[n], False)
        
        #Finding a square to bifurcate will sometimes fail if the current board is unsolvable,
        #because the options lists will empty before the board is solved and before inconistencies happen.
        #Need to avoid bifurcating these:
        if len(b_square) != 0:
        
            #Will return only solved boards:
            b_results = Bifurcate(b_square[0], b_square[1], copy_boards[n], copy_options[n])

            #Adds solved boards to the results:
            for n2 in range(len(b_results)):
                result_boards.append(b_results[n2][0])
                result_options.append(b_results[n2][1])
    
    solved = []
    
    #Turns result boards/options into tuple pairs; solved = [(board1, options1), (board2, options2) ...]
    for n in range(len(result_boards)):
        
        solved.append((result_boards[n], result_options[n]))
    
    return solved

        
#FUNCTION: SimpleSolve()
#Applies all solving functions that do not bifurcate the puzzle, until no more moves can be made
def SimpleSolve(puzzle, options):
    
    """Input: (puzzle board, options list)
    Applies all solving functions, excluding Bifurcate(), until no more moves can be made."""
    
    moves_made = 1000

    #Applies OnePossible(), OnlyPossible() and NakedGroups() functions to make moves, until no more moves can be made:
    while moves_made > 0:

        moves_made = 0

        #OnePossible
        result1 = OnePossible(puzzle, options)
        moves_made1 = result1[2]
        puzzle = result1[0]
        options = result1[1]

        #OnlyPossible
        result2 = OnlyPossible(puzzle, options)
        moves_made2 = result2[2]
        puzzle = result2[0]
        options = result2[1]

        options = NakedGroups(puzzle, options)

        moves_made = moves_made1 + moves_made2

    result = [puzzle, options]
    return result

### Run this cell to select a file:

In [4]:
#RUNS WIDGET TO SELECT A FILE
print("Please select a file as input:")
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

Please select a file as input:


SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

### Run this cell to solve the Sudoku:

In [5]:
#APPLIES FUNCTIONS TO SOLVE SUDOKU

#Opens the file in read mode:
file_path = my_button.files[0]
puzzle_file = open(file_path, "r")

#Reads the file and splits into a list of rows:
puzzle = puzzle_file.read()
puzzle = puzzle.splitlines()

#Turns every row in the list into another list of ints. Format: puzzle[row][col] = value
for x in range(len(puzzle)):
    puzzle[x] = list(puzzle[x])
    
    for y in range(len(puzzle[x])):
        puzzle[x][y] = int(puzzle[x][y])

        
#Creates a list of options for every square in the puzzle. Format: options[row][col] = list of options
#IMPORTANT NOTE: Careful when making lists equal to other lists, copies are not made and changes apply to both!
#Even when I made it equal to a copy of the list, things were going wrong.
options = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]

for x in range(len(options)):
    
    for y in range(len(options[x])):
        options[x][y] = GenerateOptions(x, y, puzzle)

#Checks if puzzle is valid, ie has no conflicts:
e_check = ErrorCheck(puzzle)
if e_check == False:
    print("There is a conflict in this puzzle! Please double check your file.")
    
#If the puzzle is valid, run the solver:
elif e_check == True:

    #Applies OnePossible(), OnlyPossible() and NakedGroups() functions to make moves, until no more moves can be made:
    updated_puzzle = SimpleSolve(puzzle, options)
    puzzle = updated_puzzle[0]
    options = updated_puzzle[1]

    #Applies Bifurcate() if puzzle is not solved:
    solve_check = CountUnsolved(puzzle)
    not_solved = False

    if solve_check > 0:

        bsquare = FindBifurcation(puzzle, options, False)

        if len(bsquare) != 0:
            solved_puzzles = Bifurcate(bsquare[0], bsquare[1], puzzle, options)

        #If no solutions:
        if len(solved_puzzles) == 0:
            not_solved = True
            print("This puzzle has no solution!")
            print()
            for x in puzzle:
                print(x)

        elif len(solved_puzzles) == 1:
            puzzle = solved_puzzles[0][0]
            options = solved_puzzles[0][1]

        #Prints the solutions if there are more than one:
        elif len(solved_puzzles) > 1:
            print("This puzzle has multiple solutions!")
            not_solved = True
            print()
            print("The solved puzzles are:")

            for n in range(len(solved_puzzles)):
                print()

                for x in solved_puzzles[n][0]:
                    print(x)

    #Prints solved puzzle if there is one solution:
    if not_solved == False:
        print()
        print("The solved puzzle is:")
        print()

        for x in puzzle:
            print(x)

        #Checks and prints how many squares are not solved
        unsolved_count = CountUnsolved(puzzle)

        print()
        print(f"{unsolved_count} squares could not be solved.")

#Closes the file:
puzzle_file.close()


The solved puzzle is:

[8, 1, 2, 7, 5, 3, 6, 4, 9]
[9, 4, 3, 6, 8, 2, 1, 7, 5]
[6, 7, 5, 4, 9, 1, 2, 8, 3]
[1, 5, 4, 2, 3, 7, 8, 9, 6]
[3, 6, 9, 8, 4, 5, 7, 2, 1]
[2, 8, 7, 1, 6, 9, 5, 3, 4]
[5, 2, 1, 9, 7, 4, 3, 6, 8]
[4, 3, 8, 5, 2, 6, 9, 1, 7]
[7, 9, 6, 3, 1, 8, 4, 5, 2]

0 squares could not be solved.
